In [1]:
# pip install lxml
# pip install html5lib
# pip install beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup
import functools
import pandas as pd

def fetch_links(base_url, page_range, link_pattern):
    links = list()
    
    for k in range(page_range[0], page_range[1]+1):
        
        page_url = base_url % k
        
        page_reponse = requests.get(
            page_url,
            headers={'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}, # Request page as Googlebot
            verify=False # ignore ssl
        )
        
        page_code = page_reponse.status_code
        
        if page_code == 200:
            
            page_html = page_reponse.text
            parser = BeautifulSoup(page_html, 'html.parser')
            
            page_links = list(map(
                lambda a: a.get('href'),
                parser.find_all('a', href=True)
            ))
            
            page_links = list(filter(
                lambda href: link_pattern in href,
                page_links
            ))
            
            print('%d links collected from page %d' % (len(page_links), k))
            
            links += page_links
        
        else:
            raise Exception('Page (%s) returned code: %d' % (page_url, page_code))
            
    return links




def parse_page_data(page_url):
    
    page_reponse = requests.get(
        page_url,
        headers={'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}, # Request page as Googlebot
        verify=False # ignore ssl
    )
    
    page_code = page_reponse.status_code
    
    if page_code == 200:

        page_html = page_reponse.text
        parser = BeautifulSoup(page_html, 'html.parser')
        data = {'page_id': page_url.split('record.php?id=')[1]}
        
        # Table
        table = parser.find('table')
        rows = table.find_all('tr')
        for row in rows:
            cols = list(map(
                lambda cell: cell.text.strip(),
                row.find_all('td')
            ))
            field_name = cols[0].strip(':').lower().replace(' ', '_')
            fiel_value = cols[1]
            data[ field_name ] = fiel_value
        
        # Narrative
        narrative = parser.find('span', {'lang': 'en-US'})
        data['narrative'] = narrative.text.strip()
        
        return data

    else:
        raise Exception('Page (%s) returned code: %d' % (page_url, page_code))



# Test
links = fetch_links(
    base_url='https://aviation-safety.net/database/events/dblist.php?Event=REO&page=%d',
    page_range=(1, 2),
    link_pattern='/database/record.php?id='
)
print( links )


data = parse_page_data('https://aviation-safety.net/database/record.php?id=19970721-2')
print(data)

C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


100 links collected from page 1


C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


100 links collected from page 2
['/database/record.php?id=19009999-01', '/database/record.php?id=19009999-51', '/database/record.php?id=19350624-0', '/database/record.php?id=19361209-0', '/database/record.php?id=19370218-0', '/database/record.php?id=19390499-0', '/database/record.php?id=19390905-0', '/database/record.php?id=19390928-0', '/database/record.php?id=19391115-0', '/database/record.php?id=19400403-0', '/database/record.php?id=19410310-1', '/database/record.php?id=19411114-0', '/database/record.php?id=19420512-1', '/database/record.php?id=19430718-2', '/database/record.php?id=19440411-2', '/database/record.php?id=19440507-2', '/database/record.php?id=19440516-0', '/database/record.php?id=19441011-3', '/database/record.php?id=19441123-0', '/database/record.php?id=19450221-0', '/database/record.php?id=19450305-1', '/database/record.php?id=19450413-6', '/database/record.php?id=19450817-1', '/database/record.php?id=19450913-0', '/database/record.php?id=19451016-0', '/database/reco

C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'page_id': '19970721-2', 'status': 'Final', 'date': 'Monday 21 July 1997', 'time': '13:49', 'type': 'de Havilland Canada DHC-6 Twin Otter 300', 'operator': 'Continental Aviation Services', 'registration': 'N776BF', 'msn': '672', 'first_flight': '1980', 'total_airframe_hrs': '12152', 'crew': 'Fatalities: 0 / Occupants: 2', 'passengers': 'Fatalities: 0 / Occupants: 0', 'total': 'Fatalities: 0 / Occupants: 2', 'aircraft_damage': 'Substantial', 'aircraft_fate': 'Repaired', 'location': 'Elko, NV ( \xa0 United States of America)', 'phase': 'Landing (LDG)', 'nature': 'Private', 'departure_airport': 'Reno/Tahoe International Airport, NV (RNO/KRNO), United States of America', 'destination_airport': 'Elko Airport, NV (EKO/KEKO), United States of America', 'narrative': 'A de Havilland DHC-6-300, N776BF, collapsed the nose gear on landing at the Elko Municipal Airport, Nevada. The aircraft was substantially damaged. Touchdown at Elko was slightly right of centerline and when the nose gear touched

In [3]:
links = fetch_links(
    base_url='https://aviation-safety.net/database/events/dblist.php?Event=REO&page=%d',
    page_range=(1, 20),
    link_pattern='/database/record.php?id='
)

print('links retrieved', len(links))

urls = list(map(
    lambda endpoint: 'https://aviation-safety.net' + endpoint,
    links
))

# Fisrt 10 urls
urls[:10]

C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


100 links collected from page 1


C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


100 links collected from page 2


C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


100 links collected from page 19


C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


23 links collected from page 20
links retrieved 1923


['https://aviation-safety.net/database/record.php?id=19009999-01',
 'https://aviation-safety.net/database/record.php?id=19009999-51',
 'https://aviation-safety.net/database/record.php?id=19350624-0',
 'https://aviation-safety.net/database/record.php?id=19361209-0',
 'https://aviation-safety.net/database/record.php?id=19370218-0',
 'https://aviation-safety.net/database/record.php?id=19390499-0',
 'https://aviation-safety.net/database/record.php?id=19390905-0',
 'https://aviation-safety.net/database/record.php?id=19390928-0',
 'https://aviation-safety.net/database/record.php?id=19391115-0',
 'https://aviation-safety.net/database/record.php?id=19400403-0']

In [4]:
# Proces one by one (Not in parallel)
import time

values = []
total = len(urls)
actual = 0

for url in urls:
    
    actual+=1
    print('Progress: %.1f%%' % (actual/total), end='\r')
    
    data = parse_page_data(url)
    values.append( data )
    
    # Wait 3 seconds to continue
    time.sleep(3)

print('Done!')

C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


C:\Users\503276344\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aviation-safety.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ConnectionError: HTTPSConnectionPool(host='aviation-safety.net', port=443): Max retries exceeded with url: /database/record.php?id=19530119-1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000253B46E27F0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [ ]:
df_aviation_safety_network = pd.DataFrame( values )
df_aviation_safety_network.to_csv('aviation-safety-network.csv')
df_aviation_safety_network.head(10)

NameError: name 'values' is not defined

In [264]:
df = pd.read_csv('aviation-safety-network.csv')

In [265]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1923 entries, 0 to 1922
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1923 non-null   int64  
 1   page_id               1923 non-null   object 
 2   status                1110 non-null   object 
 3   date                  1923 non-null   object 
 4   type                  1923 non-null   object 
 5   operator              1836 non-null   object 
 6   registration          1923 non-null   object 
 7   msn                   1877 non-null   object 
 8   first_flight          1759 non-null   object 
 9   crew                  1923 non-null   object 
 10  passengers            1923 non-null   object 
 11  total                 1923 non-null   object 
 12  aircraft_damage       1923 non-null   object 
 13  location              1923 non-null   object 
 14  phase                 1923 non-null   object 
 15  nature               

In [266]:
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0,page_id,status,date,type,operator,registration,msn,first_flight,crew,passengers,total,aircraft_damage,location,phase,nature,departure_airport,destination_airport,narrative,time,engines,collision_casualties,aircraft_fate,crash_site_elevation,flightnumber,total_airframe_hrs,ground_casualties,operating_for,leased_from,cycles,operated_by,on_behalf_of
0,0,19009999-01,NaN,date unk.,Antonov An-12BK,Soviet Air Force,registration unknown,9346609,1969,Fatalities: / Occupants:,Fatalities: / Occupants:,Fatalities: / Occupants:,Damaged beyond repair,Tiksi Airport (IKS) ( Russia),Unknown (UNK),Military,?,?,The aircraft was damaged beyond repair after i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,19009999-51,NaN,date unk.,Britten-Norman BN-2A Islander,MD Air Services,N67HA,241,1973,Fatalities: 0 / Occupants:,Fatalities: 0 / Occupants:,Fatalities: 0 / Occupants:,Damaged beyond repair,Nassau International Airport (NAS) ( Bahamas),Landing (LDG),Unknown,?,"Nassau International Airport (NAS/MYNN), Bahamas",The Islander was damaged beyond repair after o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,19350624-0,NaN,Monday 24 June 1935,Ford 5-AT-B Tri-Motor,Servicio Aéreo Colombiano - SACO,F-31,5-AT-006,1928-11-22 (6 years 7 months),Fatalities: 2 / Occupants: 2,Fatalities: 8 / Occupants: 11,Fatalities: 10 / Occupants: 13,Destroyed,Medellín-Enrique Olaya Herrera Airport (EOH) (...,Takeoff (TOF),Domestic Scheduled Passenger,Medellín-Enrique Olaya Herrera Airport (EOH/SK...,"Bogotá-Techo Airport, Colombia",Two Ford 5-AT Tri-Motor passenger planes were ...,14:51,3 Pratt & Whitney R-1340 Wasp,Fatalities: 7,Written off (damaged beyond repair),1506 m (4941 feet) amsl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,19361209-0,NaN,Wednesday 9 December 1936,Douglas DC-2-115E,KLM Royal Dutch Airlines,PH-AKL,1358/F18,1935,Fatalities: 3 / Occupants: 4,Fatalities: 12 / Occupants: 13,Fatalities: 15 / Occupants: 17,Destroyed,London-Croydon Airport ( United Kingdom),Initial climb (ICL),International Scheduled Passenger,"London-Croydon Airport (-/-), United Kingdom",Amsterdam-Schiphol Municipal Airport (AMS/EHAM...,"The DC-2, named ""Lijster"", operated on a KLM p...",10:30,NaN,NaN,Written off (damaged beyond repair),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,19370218-0,NaN,Thursday 18 February 1937,Douglas DC-2-112,Eastern Air Lines,NC13734,1260,1934,Fatalities: 0 / Occupants: 3,Fatalities: 0 / Occupants: 7,Fatalities: 0 / Occupants: 10,Damaged beyond repair,"Atlanta Municipal Airport, GA (ATL) ( United...",Landing (LDG),Domestic Scheduled Passenger,"Chicago Municipal Airport, IL (MDW/KMDW), Unit...","Atlanta Municipal Airport, GA (ATL/KATL), Unit...",The aircraft ran against an embankment just af...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
import re
p = re.compile("(?<=\()[^)]*(?=\))")
txt = 'Los Angeles Airport, CA (LAX) ( Â  United States of America)'
m = p.match(txt)

In [228]:
x = re.findall("\(([A-Z)]{3})\)", txt)
x[0]

'SYD'

In [226]:
len(x)

1

In [128]:
txt = 'Los Angeles Airport, CA  ( Â  United States of America)'
y = re.findall("\((.*?)\)", txt)
print(y)

# /a([\s\S]*)

[' Â\xa0 United States of America']


In [190]:
txt = 'Sydney-Mascot Airport, NSW (SYD) ( Â  Australia)'
y = re.findall("\(([^)]*)\)[^(]*$", txt)
y

# /a([\s\S]*)

[' Â\xa0 Australia']

[' Â\xa0 Australia']

In [197]:
y[0][3:]

' Australia'

In [186]:
len(y[3])

1

In [255]:
def test(txt_string): 
    _ = re.findall("\(([A-Z)]{3})\)", txt_string)

    # _.replace("'","")
    # _.replace('[','')
    # _.replace(']','')
    return _

In [256]:
df['regex_iata2'] = df.apply(lambda x: test(x['location']).str.get(0), axis=1)

AttributeError: 'list' object has no attribute 'str'

In [268]:
df['regex_iata'] =  df['location'].apply(lambda x: re.findall("\(([A-Z)]{3})\)", x)).str.get(0)
# ser =  df['location'].apply(lambda x: re.findall("\(([A-Z)]{3})\)", x)).str.get(0)

In [269]:
df['regex_country'] =  df['location'].apply(lambda x: re.findall("\(([^)]*)\)[^(]*$", x)).str.get(0)

In [270]:
df.head()

,Unnamed: 0,page_id,status,date,type,operator,registration,msn,first_flight,crew,passengers,total,aircraft_damage,location,phase,nature,departure_airport,destination_airport,narrative,time,engines,collision_casualties,aircraft_fate,crash_site_elevation,flightnumber,total_airframe_hrs,ground_casualties,operating_for,leased_from,cycles,operated_by,on_behalf_of,regex_iata,regex_country
0,0,19009999-01,NaN,date unk.,Antonov An-12BK,Soviet Air Force,registration unknown,9346609,1969,Fatalities: / Occupants:,Fatalities: / Occupants:,Fatalities: / Occupants:,Damaged beyond repair,Tiksi Airport (IKS) ( Russia),Unknown (UNK),Military,?,?,The aircraft was damaged beyond repair after i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IKS,Russia
1,1,19009999-51,NaN,date unk.,Britten-Norman BN-2A Islander,MD Air Services,N67HA,241,1973,Fatalities: 0 / Occupants:,Fatalities: 0 / Occupants:,Fatalities: 0 / Occupants:,Damaged beyond repair,Nassau International Airport (NAS) ( Bahamas),Landing (LDG),Unknown,?,"Nassau International Airport (NAS/MYNN), Bahamas",The Islander was damaged beyond repair after o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAS,Bahamas
2,2,19350624-0,NaN,Monday 24 June 1935,Ford 5-AT-B Tri-Motor,Servicio Aéreo Colombiano - SACO,F-31,5-AT-006,1928-11-22 (6 years 7 months),Fatalities: 2 / Occupants: 2,Fatalities: 8 / Occupants: 11,Fatalities: 10 / Occupants: 13,Destroyed,Medellín-Enrique Olaya Herrera Airport (EOH) (...,Takeoff (TOF),Domestic Scheduled Passenger,Medellín-Enrique Olaya Herrera Airport (EOH/SK...,"Bogotá-Techo Airport, Colombia",Two Ford 5-AT Tri-Motor passenger planes were ...,14:51,3 Pratt & Whitney R-1340 Wasp,Fatalities: 7,Written off (damaged beyond repair),1506 m (4941 feet) amsl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EOH,Colombia
3,3,19361209-0,NaN,Wednesday 9 December 1936,Douglas DC-2-115E,KLM Royal Dutch Airlines,PH-AKL,1358/F18,1935,Fatalities: 3 / Occupants: 4,Fatalities: 12 / Occupants: 13,Fatalities: 15 / Occupants: 17,Destroyed,London-Croydon Airport ( United Kingdom),Initial climb (ICL),International Scheduled Passenger,"London-Croydon Airport (-/-), United Kingdom",Amsterdam-Schiphol Municipal Airport (AMS/EHAM...,"The DC-2, named ""Lijster"", operated on a KLM p...",10:30,NaN,NaN,Written off (damaged beyond repair),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United Kingdom
4,4,19370218-0,NaN,Thursday 18 February 1937,Douglas DC-2-112,Eastern Air Lines,NC13734,1260,1934,Fatalities: 0 / Occupants: 3,Fatalities: 0 / Occupants: 7,Fatalities: 0 / Occupants: 10,Damaged beyond repair,"Atlanta Municipal Airport, GA (ATL) ( United...",Landing (LDG),Domestic Scheduled Passenger,"Chicago Municipal Airport, IL (MDW/KMDW), Unit...","Atlanta Municipal Airport, GA (ATL/KATL), Unit...",The aircraft ran against an embankment just af...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,United States of America


In [251]:
ser

0       [IKS]
1       [NAS]
2       [EOH]
3          []
4       [ATL]
        ...  
1918    [GYN]
1919    [MNI]
1920       []
1921    [MCO]
1922    [VGT]
Name: location, Length: 1923, dtype: object

In [ ]:
txt[0]

'S'

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1923 entries, 0 to 1922
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1923 non-null   int64  
 1   page_id               1923 non-null   object 
 2   status                1110 non-null   object 
 3   date                  1923 non-null   object 
 4   type                  1923 non-null   object 
 5   operator              1836 non-null   object 
 6   registration          1923 non-null   object 
 7   msn                   1877 non-null   object 
 8   first_flight          1759 non-null   object 
 9   crew                  1923 non-null   object 
 10  passengers            1923 non-null   object 
 11  total                 1923 non-null   object 
 12  aircraft_damage       1923 non-null   object 
 13  location              1923 non-null   object 
 14  phase                 1923 non-null   object 
 15  nature               

In [151]:
df.regex_country.value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[   United States of America]    389
[   Brazil]                       97
[   Russia]                       95
[   Indonesia]                    82
[   Canada]                       75
                                ... 
[   Samoa]                         1
[   Cuba]                          1
[   Mauritania]                    1
[   Paraguay]                      1
[   Vanuatu]                       1
Name: regex_country, Length: 169, dtype: int64

In [271]:
df.to_csv('regex_test.csv')

In [ ]:
df